In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import Birch
from sklearn.decomposition import PCA
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D

In [128]:
df=pd.read_excel('df0.xlsx')
df.drop(0,axis=0,inplace=True)

cols=[ 'Age', 'BCLOT', 'A/G RAT', 'DBIL', 'IBIL',
       'TBIL', 'TGLO', 'TREP', 'ALB', 'GGT', 'SGOT', 'SGPT','ALKPO4']
for col in cols:
    df[col]= df[col].apply(pd.to_numeric)
meanAge=df['Age'].mean()
df['Age'].fillna(value=meanAge,inplace=True)

In [6]:
df.dtypes

Age        float64
Gender      object
BCLOT      float64
A/G RAT    float64
DBIL       float64
IBIL       float64
TBIL       float64
TGLO       float64
TREP       float64
ALB        float64
GGT        float64
SGOT       float64
SGPT       float64
ALKPO4     float64
dtype: object

In [129]:
n=len(df.index)
print(n)

327155


In [130]:
def checkGGT(gender,ggt):
    if(gender=='M'):
        if(ggt>=15 and ggt<=73):
            return 0
        else:
            return 1
    else:
        if(ggt>=12 and ggt<=43):
            return 0
        else:
            return 1

In [9]:
def checkSGOT(gender,sgot):
    if(gender=='M'):
        if(sgot>=17 and sgot<=59):
            return 0
        else:
            return 1
    else:
        if(sgot>=14 and sgot<=36):
            return 0
        else:
            return 1

In [131]:
def checkSGPT(gender,sgpt):
    if(gender=='M'):
        if(sgpt>=21 and sgpt<=72):
            return 0
        else:
            return 1
    else:
        if(sgpt>=9 and sgpt<=53):
            return 0
        else:
            return 1

In [132]:
def checkTBIL(tbil):
    
        if(tbil>=0.3 and tbil<=1.2):
            return 0
        else:
            return 1
    

In [133]:
def checkTGLO(tglo):
    
        if(tglo>=2.0 and tglo<=3.0):
            return 0
        else:
            return 1
    

In [134]:
def checkTREP(trep):
    
        if(trep>=6.0 and trep<=8.5):
            return 0
        else:
            return 1
    

In [135]:
def isAgrat(agrat):
  if (agrat<1.5) or (agrat>2.5):
    return 1
  return 0

def isAlb(alb):
  if (alb<3.5) or (alb>5.2):
    return 1
  return 0

def isBclot(bclot, age):
  if (age<=18):
    if (bclot<200):
      return 0
    return 1


  else:
    if (bclot < 240):
      return 0
    return 1

def isDbil(dbil):
  if (dbil<0) or (dbil>0.2):
    return 1
  return 0

def isIbil(ibil):
  if (ibil<0) or (ibil>0.8):
    return 1
  return 0

def isAlk(alk, age, gender):
  if (age<=20):
    if (alk<54) or (alk>369):
      return 1
    return 0
  elif(age<=60):
    if(gender=='M'):
      if (alk<53) or (alk>128):
        return 1
      return 0
    else:
      if (alk<42) or (alk>98):
        return 1
      return 0
  else:
    if(gender=='M'):
      if (alk<56) or (alk>119):
        return 1
      return 0
    else:
      if (alk<53) or (alk>141):
        return 1
      return 0

In [136]:
abnormal_count={'BCLOT':0,'A/G RAT':0,'DBIL':0,'IBIL':0,'TBIL':0,'TGLO':0,'TREP':0,'ALB':0,'GGT':0,'SGOT':0,'SGPT':0,'ALKPO4':0}

In [137]:
def checkRow(i,arr_index,abnormal):
    row=df.iloc[i]
    age=row['Age']
    gender=row['Gender']
    bclot=row['BCLOT']
    agr=row['A/G RAT']
    dbil=row['DBIL']
    ibil=row['IBIL']
    tbil=row['TBIL']
    tglo=row['TGLO']
    trep=row['TREP']
    alb=row['ALB']
    ggt=row['GGT']
    sgot=row['SGOT']
    sgpt=row['SGPT']
    alkpo4=row['ALKPO4']
    
    if(isBclot(bclot,age)==1):
        abnormal_count['BCLOT']+=1
        abnormal.append(1)
        return 1
    if(isAgrat(agr)==1):
        abnormal_count['A/G RAT']+=1
        abnormal.append(1)
        return 1
    if(isDbil(dbil)==1):
        abnormal_count['DBIL']+=1
        abnormal.append(1)
        return 1
    if(isIbil(ibil)==1):
        abnormal_count['IBIL']+=1
        abnormal.append(1)
        return 1
    if(checkTBIL(tbil)==1):
        abnormal_count['TBIL']+=1
        abnormal.append(1)
        return 1
    if(checkTGLO(tglo)==1):
        abnormal_count['TGLO']+=1
        abnormal.append(1)
        return 1
    if(checkTREP(trep)==1):  
        abnormal_count['TREP']+=1
        abnormal.append(1)
        return 1
    if(isAlb(alb)==1 ):
        abnormal_count['ALB']+=1
        abnormal.append(1)
        return 1
    if(checkGGT(gender,ggt)==1):
        abnormal_count['GGT']+=1
        abnormal.append(1)
        return 1
    if(checkSGOT(gender,sgot)==1):
        abnormal_count['SGOT']+=1
        abnormal.append(1)
        return 1
    if(checkSGPT(gender,sgpt)==1): 
        abnormal_count['SGPT']+=1
        abnormal.append(1)
        return 1
    if(isAlk(alkpo4,age,gender)==1):
        abnormal_count['ALKPO4']+=1
        abnormal.append(1)
        return 1
    abnormal.append(0)
    
    

In [138]:
abnormal=[]
arr_index=-1
n=len(df.index)
for i in range(0,n):
    checkRow(i,arr_index,abnormal)

In [142]:
abnormal.count(1)

282987

In [140]:
print(abnormal_count)

{'BCLOT': 43678, 'A/G RAT': 95807, 'DBIL': 95275, 'IBIL': 220, 'TBIL': 4196, 'TGLO': 5031, 'TREP': 722, 'ALB': 243, 'GGT': 14109, 'SGOT': 9678, 'SGPT': 5314, 'ALKPO4': 8714}


In [24]:
abnormal.count(0)

44168

In [34]:
df_info=df_info.dropna()

In [35]:
df_coded=df_info

In [36]:
df_coded.drop(331566,axis=0,inplace=True)

C:\Users\delwy\AppData\Local\Temp\ipykernel_16144\817448536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coded.drop(331566,axis=0,inplace=True)


In [37]:
df_coded

,Unique ID,Age,Gender,BCLOT,A/G RAT,DBIL,IBIL,TBIL,TGLO,TREP,ALB,GGT,SGOT,SGPT,ALKPO4
1,1.0,73.0,M,358,1.15,0.4,0.8,1.2,3.4,7.3,3.9,25,26,32,58
2,2.0,22.0,M,172,1.85,0.42,0.68,1.1,2.42,6.9,4.48,20,27,30,61
3,3.0,36.0,F,166,1.48,0.11,0.2,0.31,2.82,7,4.18,16,28,36,97
4,4.0,52.0,F,213,1.26,0.3,0.3,0.6,3.5,7.9,4.4,16,20,28,89
5,5.0,35.0,F,187,1.24,0.11,0.17,0.28,3.48,7.8,4.32,39,20,18,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331561,331563.0,38.0,M,206,1.75,0.2,0.47,0.67,2.36,6.5,4.14,46,22,50,98
331562,331564.0,60.0,F,142,1.62,0.17,0.36,0.53,2.6,6.8,4.2,20,21,22,69
331563,331565.0,27.0,M,166,1.36,0.34,0,0.34,3.9,9.2,5.3,164,37,39,123
331564,331566.0,42.0,F,219,1.41,0.1,0.24,0.34,2.94,7.1,4.16,25,40,60,65


In [99]:
cols=['Unique ID', 'Age', 'BCLOT', 'A/G RAT', 'DBIL', 'IBIL',
       'TBIL', 'TGLO', 'TREP', 'ALB', 'GGT', 'SGOT', 'SGPT', 'ALKPO4']

In [100]:
for col in cols:
    df_info[col]= df_info[col].apply(pd.to_numeric)

In [22]:
cols=['BCLOT', 'A/G RAT', 'DBIL', 'IBIL',
       'TBIL', 'TGLO', 'TREP', 'ALB', 'GGT', 'SGOT', 'SGPT', 'ALKPO4']
for col in cols:
    print(col+'Normal Readings:'+str(df[col].value_counts()[0.0]))

BCLOTNormal Readings:42
A/G RATNormal Readings:42
DBILNormal Readings:81
IBILNormal Readings:1387
TBILNormal Readings:43
TGLONormal Readings:42
TREPNormal Readings:30
ALBNormal Readings:49
GGTNormal Readings:43
SGOTNormal Readings:53
SGPTNormal Readings:51
ALKPO4Normal Readings:53
